# Calls between two personas
### The objective of the code challenge is to get the number of calls between two personas and the total duration.<br>

> *Is not important who dial the call and who received the call


In [0]:
from pyspark.sql.functions import to_date, col, lag, lead, when, sum, first, last, array, array_sort, count, get
from pyspark.sql.window import Window

In [0]:
data = [
  (50,20,60),
  (20,30,25),
  (40,80,13),
  (90,70,45),
  (70,90,30),
  (80,40,12),
  (20,50,9),
  (10,60,3),
  (40,80,7),
  (60,70,30),
  (90,70,20),
  (70,60,9)
  ]

df = spark.createDataFrame(data,['from_id','to_id','duration'])
display(df)

from_id,to_id,duration
50,20,60
20,30,25
40,80,13
90,70,45
70,90,30
80,40,12
20,50,9
10,60,3
40,80,7
60,70,30


## Solution 1

In [0]:
df_array = df.withColumn('ids', array_sort(array('from_id','to_id')))

In [0]:
df_array = df_array.groupBy(get(df_array.ids,0).alias('person1'),get(df_array.ids,1).alias('person2')).agg(count('duration').alias('call_count'),sum('duration').alias('total_duration')).orderBy('person1','person2')
display(df_array)

person1,person2,call_count,total_duration
10,60,1,3
20,30,1,25
20,50,2,69
40,80,3,32
60,70,2,39
70,90,3,95


## Solution 2

In [0]:
df_case = df.withColumn('person1', when(col('from_id') > col('to_id'),col('to_id')).otherwise(col('from_id')) ).withColumn('person2', when(col('from_id') < col('to_id'),col('to_id')).otherwise(col('from_id')) )
df_case.display()

from_id,to_id,duration,person1,person2
50,20,60,20,50
20,30,25,20,30
40,80,13,40,80
90,70,45,70,90
70,90,30,70,90
80,40,12,40,80
20,50,9,20,50
10,60,3,10,60
40,80,7,40,80
60,70,30,60,70


In [0]:
df_case = df_case.groupBy('person1','person2').agg(count('duration').alias('call_count'),sum('duration').alias('total_duration')).orderBy('person1','person2')
df_case.display()

person1,person2,call_count,total_duration
10,60,1,3
20,30,1,25
20,50,2,69
40,80,3,32
60,70,2,39
70,90,3,95
